<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex12notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex12notebookA

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

---
## 判別分析 (2)
---



In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy のもろもろ
import scipy as sp
from scipy.stats import norm, multivariate_normal

# scikit-learn のもろもろ
from sklearn.datasets import load_iris
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

# 文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 線形判別分析と二次判別分析



$D$次元のデータを「クラス1」と「クラス2」の2クラスに分類する問題に対して，前回説明した判別分析は，次のことを行うものでした．

1. それぞれのクラスのデータに正規分布を当てはめる．すなわち，クラス1の正規分布の平均 $\pmb{\mu}_1$ と分散共分散行列 $\Sigma_1$ およびクラス2の正規分布の平均 $\pmb{\mu}_2$ と分散共分散行列 $\Sigma_2$ を推定する．
1. 判別関数 $h(\pmb{x})$ を次式で定める（$\ell_1(\pmb{x})$ および $\ell_2(\pmb{x})$ は，クラス1の正規分布およびクラス2の正規分布に対する $\pmb{x}$ の尤度 ）．
$$
\begin{aligned}
h(\pmb{x}) &= \log{\ell_1(\pmb{x})} - \log{\ell_2(\pmb{x})} & (1) \\
&=  -\frac{1}{2} (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma_1^{-1}(\pmb{x}-\pmb{\mu}_1) + \frac{1}{2} (\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma_2^{-1}(\pmb{x}-\pmb{\mu}_2) + \frac{1}{2}\log{\frac{|\Sigma_2|}{|\Sigma_1|}} & (2)
\end{aligned}
$$
1. データ $\pmb{x}$ のクラスを， $h(\pmb{x}) > 0$ ならばクラス1，$h(\pmb{x}) < 0$ ならばクラス2と予測する．

実は，この判別分析の手法には，**二次判別分析** (Quadratic Discriminant Analysis, QDA) という名前がついています（注）．
二次判別分析は，クラスごとに分散共分散行列を推定しなければならない点や，判別関数が複雑になる点で，計算の手間の多い手法です．この計算の負担を軽減するため，分散共分散行列の扱いを簡略化し，より効率的に計算を行う手法として，**線形判別分析** (Linear Discriminant Analysis, LDA) が考案されています（注）．次節では，2クラスの線形判別分析について説明します

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 二つの判別分析の名前の由来（なぜ「二次」なのか，何が「線形」なのか）については後述します．
</span>

---
### 線形判別分析

#### 線形判別分析とは

**線形判別分析** (Linear Discriminant Analysis, LDA)（注） は，二次判別分析に「各クラスの正規分布は全て同一の分散共分散行列を持つ（平均のみが異なる）」という制約条件を加えて計算を簡略化した判別分析手法です．

2クラスの二次判別分析の場合，
クラス1の正規分布の平均 $\pmb{\mu}_1$，分散共分散行列 $\Sigma_1$，クラス2の正規分布の平均 $\pmb{\mu}_2$，分散共分散行列 $\Sigma_2$ の4つを求めていましたが，線形判別分析では，$\Sigma_1 = \Sigma_2$ と仮定します．この場合，$\Sigma = \Sigma_1 = \Sigma_2$ とおくと，式(2)の第3項は $\frac{1}{2}\log{\frac{|\Sigma|}{|\Sigma|}} = \log 1 = 0$ となるので，判別関数は次の通りとなります．

$$
\begin{aligned}
h(\pmb{x}) &= -\frac{1}{2} (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu}_1) + \frac{1}{2} (\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu}_2) & (3)\\
&= \frac{d_{2}^2(\pmb{x}) - d_{1}^2(\pmb{x})}{2} & (4)
\end{aligned}
$$

ただし，$d_{1}(\pmb{x}), d_{2}(\pmb{x})$ はそれぞれ，平均 $\pmb{\mu}_1$ 分散共分散行列 $\Sigma$ の正規分布に対するマハラノビス距離，平均 $\pmb{\mu}_2$ 分散共分散行列 $\Sigma$ の正規分布に対するマハラノビス距離です．


<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 線形判別分析は，Ronald Fisher が最初に研究したと考えられており，<i>Fisher's Linear Discriminant</i> と呼ばれることもあります．Ronald Fisher は，前回の notebookA の「Fisher のアヤメのデータ」の節で紹介した，著名な統計学者です．
</span>

式(3)の判別関数は，以下に示すように，変形していくと $\pmb{x}$ の一次式となっています．

$$
\begin{aligned}
h(\pmb{x}) &= \frac{1}{2}\left(
(\pmb{x}-\pmb{\mu}_2)^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu}_2) - (\pmb{x}-\pmb{\mu}_1)^{\top}\Sigma^{-1}(\pmb{x}-\pmb{\mu}_1)  \right)\\
&= \frac{1}{2}\left( \pmb{x}^{\top}\Sigma^{-1}\pmb{x} - \pmb{x}^{\top}\Sigma^{-1}\pmb{\mu}_2 - \pmb{\mu}_2^{\top}\Sigma^{-1}\pmb{x} + \pmb{\mu}_2^{\top}\Sigma^{-1}\pmb{\mu}_2 -\left( \pmb{x}^{\top}\Sigma^{-1}\pmb{x} - \pmb{x}^{\top}\Sigma^{-1}\pmb{\mu}_1 - \pmb{\mu}_1^{\top}\Sigma^{-1}\pmb{x} + \pmb{\mu}_1^{\top}\Sigma^{-1}\pmb{\mu}_1\right) \right)\\
&= \frac{1}{2}\left( -2\pmb{\mu}_2^{\top}\Sigma^{-1}\pmb{x} + \pmb{\mu}_2^{\top}\Sigma^{-1}\pmb{\mu}_2 + 2\pmb{\mu}_1^{\top}\Sigma^{-1}\pmb{x} - \pmb{\mu}_1^{\top}\Sigma^{-1}\pmb{\mu}_1 \right)\\
&= (\pmb{\mu}_1 - \pmb{\mu}_2)^{\top}\Sigma^{-1}\pmb{x} - \frac{1}{2}(\pmb{\mu}_1 - \pmb{\mu}_2)^{\top}\Sigma^{-1}(\pmb{\mu}_1+\pmb{\mu}_2)\\
&= (\pmb{\mu}_1 - \pmb{\mu}_2)^{\top}\Sigma^{-1}\left( \pmb{x} - \frac{\pmb{\mu}_1+\pmb{\mu}_2}{2} \right) \qquad (5)
\end{aligned}
$$

$h(\pmb{x}) = 0 \Leftrightarrow \pmb{x} = \frac{\pmb{\mu}_1+\pmb{\mu}_2}{2}$ ですので，クラス1とクラス2を分ける境界は，2つの平均の中点 $\frac{\pmb{\mu}_1+\pmb{\mu}_2}{2}$ を通り，ベクトル $\Sigma^{-1}(\pmb{\mu}_1 - \pmb{\mu}_2)$ に垂直な 平面（$(D-1)$次元平面，$D=2$のときは直線）となります（注）．
このことに由来して，線形判別分析は「線形」判別分析と呼ばれます．また，線形判別分析における判別関数は「線形判別関数」と呼ばれます．

一方，二次判別分析の判別関数（式(2)の $h(\pmb{x})$）を $0$ とおいて得られる2クラスの境界は，二次曲面（$D=2$のときは二次曲線，すなわち，楕円 or 放物線 or 双曲線）となっています．そのため，二次判別分析は「二次」判別分析と呼ばれます．また，二次判別分析における判別関数は「二次判別関数」と呼ばれます．

<br>
<hr width="50%" align="left">
<span style="font-size: 75%">
※注: 高校のベクトルの項で学ぶ直線の方程式の話の3次元以上への一般化．$\pmb{n}^{\top}(\pmb{x}-\pmb{p}) = \pmb{n}\cdot (\pmb{x}-\pmb{p}) = 0$ を満たす点 $\pmb{x}$ は，点 $\pmb{p}$ を通り $\pmb{n}$ を法線ベクトルとする平面上にあります．
</span>


#### 1次元の線形判別分析の例

クラス1, クラス2の2クラスデータに線形判別分析を適用する場合，クラス1の平均を $\mu_1$，クラス2の平均を $\mu_2$，両クラス共通の分散を $\sigma^2$ とおくと，

$$
d_1^2(x) = \frac{(x-\mu_1)^2}{\sigma^2}, \quad d_2^2(x) = \frac{(x-\mu_2)^2}{\sigma^2} \qquad (6)
$$

より，判別関数は次の通りとなります．

$$
\begin{aligned}
h(x) &= \frac{1}{2\sigma^2}\left( (x-\mu_2)^2 - (x-\mu_1)^2 \right) = \frac{1}{2\sigma^2}\left( x^2 - 2\mu_2 x + \mu_2^2 - (x^2 - 2\mu_1 x + \mu_1^2) \right)\\
&= \frac{1}{2\sigma^2}\left( 2(\mu_1 - \mu_2)x - (\mu_1^2 - \mu_2^2) \right)
= \frac{\mu_1 - \mu_2}{2\sigma^2}\left( 2x - (\mu_1 + \mu_2)\right) \\
&= \frac{\mu_1 - \mu_2}{\sigma^2}\left( x - \frac{\mu_1 + \mu_2}{2}\right) & (7)
\end{aligned}
$$

$h(x) = 0 \Leftrightarrow x = \frac{\mu_1+\mu_2}{2}$ ですので，2つの平均の中点がクラス1とクラス2を分ける境界となります．

前回と同じ「人間」vs「ほげ星人」のデータのうち身長の方のみを使い，線形判別分析と二次判別分析の違いを見てみましょう．

In [ ]:
# 人間 vs ほげ星人
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/humanvshoge.csv'
dfHoge = pd.read_csv(URL)
#dfHoge

# X_Human が「人間」の身長，X_Hoge が「ほげ星人」の身長
X_Human = dfHoge.loc[dfHoge['Class'] == 'Human', 'Height'].to_numpy()
X_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  'Height'].to_numpy()

次のセルで二次判別分析を実行します．

In [ ]:
##### 二次判別分析（1次元） #####

# クラス1，クラス2それぞれの平均を推定
mu1 = np.mean(X_Human)
mu2 = np.mean(X_Hoge)
print(f'mu1 = {mu1:.2f}, mu2 = {mu2:.2f}')

# クラス1, クラス2それぞれの分散を推定
s1 = np.var(X_Human)
s2 = np.var(X_Hoge)
print(f's1 = {s1:.2f}, s2 = {s2:.2f}')

# 二次判別関数の定義（ここでは scipy.stats.norm.logpdf を利用）
def hQ(x, mu1, s1, mu2, s2):
    return norm.logpdf(x, loc=mu1, scale=np.sqrt(s1)) - norm.logpdf(x, loc=mu2, scale=np.sqrt(s2))

次のセルで線形判別分析を実行します．2クラス共通の分散は，標本分散を用いて次のように推定します．

$$
\hat{\sigma}^2 = \frac{1}{\mbox{データの総数}}\left( \sum_{n:\mbox{人間}} (x_n - \mu_1)^2 + \sum_{n:\mbox{ほげ星人}} (x_n - \mu_2)^2 \right) \qquad (8)
$$


In [ ]:
##### 線形判別分析（1次元） #####

# クラス1，クラス2それぞれの平均を推定
mu1 = np.mean(X_Human)
mu2 = np.mean(X_Hoge)
print(f'mu1 = {mu1:.2f}, mu2 = {mu2:.2f}')

# 共通の分散
XX = np.hstack((X_Human - mu1, X_Hoge - mu2))
s = np.mean(XX**2)
print(f's = {s:.2f}')

# 線形判別関数の定義
def hL(x, mu1, mu2, s):
    return (mu1 - mu2) / s * (x - (mu1 + mu2) / 2)

それぞれの結果を可視化してみると，次のようになります．

In [ ]:
### 身長のヒストグラム，推定された正規分布の確率密度関数，判別境界
#
xmin, xmax = 80, 220
bins = np.linspace(xmin, xmax, 28)
xx = np.linspace(xmin, xmax, 100)

fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# 二次判別分析
ax[0].hist(X_Human, bins=bins, alpha=0.8, label='Human', density=True)
ax[0].hist(X_Hoge,  bins=bins, alpha=0.8, label='Hoge',  density=True)
px1 = norm.pdf(xx, loc=mu1, scale=np.sqrt(s1))
ax[0].plot(xx, px1, linewidth=2, color='blue')
px2 = norm.pdf(xx, loc=mu2, scale=np.sqrt(s2))
ax[0].plot(xx, px2, linewidth=2, color='red')
hx = lambda x: hQ(x, mu1, s1, mu2, s2)
pQ = sp.optimize.brentq(hx, xmin, xmax)
ax[0].axvline(pQ, linewidth=2, color='purple', label=f'h(x) = 0')
ax[0].set_xlim(xmin, xmax)
ax[0].set_title('QDA')
ax[0].legend()

# 線形判別分析
ax[1].hist(X_Human, bins=bins, alpha=0.8, label='Human', density=True)
ax[1].hist(X_Hoge,  bins=bins, alpha=0.8, label='Hoge',  density=True)
px1 = norm.pdf(xx, loc=mu1, scale=np.sqrt(s))
ax[1].plot(xx, px1, linewidth=2, color='blue')
px2 = norm.pdf(xx, loc=mu2, scale=np.sqrt(s))
ax[1].plot(xx, px2, linewidth=2, color='red')
pL = (mu1 + mu2) / 2
ax[1].axvline(pL, linewidth=2, color='purple', label=f'h(x) = 0')
ax[1].set_xlim(xmin, xmax)
ax[1].set_title('LDA')
ax[1].legend()

fig.tight_layout()
plt.show()

print(f'二次判別の判別境界: x = {pQ:.2f}     線形判別の判別境界: x = {pL:.2f}')

左が二次判別分析，右が線形判別分析の結果です．線形判別分析では，2クラスの正規分布の分散が等しいと仮定しているため，確率密度関数の形が等しくなっています．
判別境界（$h(x) = 0$ となる $x$ の値）も異なっています．

#### 2次元の線形判別分析の例

「人間」vs「ほげ星人」のデータの2変数（身長，体重）を両方用いて2次元の判別分析を行ってみましょう．

In [ ]:
# 人間 vs ほげ星人のデータを格納した2次元配列をつくる
X2_Human = dfHoge.loc[dfHoge['Class'] == 'Human', ['Height', 'Weight']].to_numpy()
X2_Hoge  = dfHoge.loc[dfHoge['Class'] == 'Hoge',  ['Height', 'Weight']].to_numpy()

次のセルで二次判別分析を実行します．

In [ ]:
##### 二次判別分析（2次元） #####

# クラス1，クラス2それぞれの平均を推定
mu1 = np.mean(X2_Human, axis=0)
mu2 = np.mean(X2_Hoge, axis=0)
print(f'mu1 = {mu1}, mu2 = {mu2}')

# クラス1, クラス2それぞれの分散共分散行列を推定
XX = X2_Human - mu1
cov1 = XX.T @ XX / XX.shape[0]
print('cov1 = ')
print(cov1)
XX = X2_Hoge - mu2
cov2 = XX.T @ XX / XX.shape[0]
print('cov2 = ')
print(cov2)

次のセルで線形判別分析を実行します．2クラス共通の分散共分散行列の求め方の説明は省略しますが，考え方は1次元の場合と同様です．


In [ ]:
##### 線形判別分析（2次元） #####

# 平均の表示（↑で求めたもの）
print(f'mu1 = {mu1}, mu2 = {mu2}')

# 平均を引いたデータ行列をつくる
XX = np.vstack((X2_Human - mu1, X2_Hoge - mu2))
# それを用いて共通の分散共分散行列を求める
cov = XX.T @ XX / XX.shape[0]
print('cov = ')
print(cov)

次のセルで判別関数の値を求める関数を定義します．

In [ ]:
# 判別関数の定義（scipy.stats.multivariate_normal.logpdf を利用）
def h2(X, mu1, cov1, mu2, cov2):
    L1 = multivariate_normal.logpdf(X, mean=mu1, cov=cov1)
    L2 = multivariate_normal.logpdf(X, mean=mu2, cov=cov2)
    return L1 - L2

二次判別・線形判別それぞれの結果を可視化してみると，次のようになります．

In [ ]:
##### 判別分析の結果を可視化する
#
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
xmin, xmax = 50, 250
ymin, ymax = 0, 150
xx, yy = np.mgrid[xmin:xmax:0.1, ymin:ymax:0.1]

# 二次判別分析
ax[0].scatter(X2_Human[:, 0], X2_Human[:, 1], label='Human')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu1, cov=cov1)
ax[0].contour(xx, yy, zz, colors='blue')
ax[0].scatter(X2_Hoge[:, 0], X2_Hoge[:, 1], label='Hoge')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu2, cov=cov2)
ax[0].contour(xx, yy, zz, colors='red')
zz = (h2(np.dstack((xx, yy)), mu1, cov1, mu2, cov2) < 0).astype(float)
ax[0].contourf(xx, yy, zz, cmap='bwr', alpha=0.2)
ax[0].set_xlim(xmin, xmax)
ax[0].set_xlabel('Height')
ax[0].set_ylim(ymin, ymax)
ax[0].set_ylabel('Weight')
ax[0].set_aspect('equal')
ax[0].set_title('QDA')
ax[0].legend()

# 線形判別分析
ax[1].scatter(X2_Human[:, 0], X2_Human[:, 1], label='Human')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu1, cov=cov)
ax[1].contour(xx, yy, zz, colors='blue')
ax[1].scatter(X2_Hoge[:, 0], X2_Hoge[:, 1], label='Hoge')
zz = multivariate_normal.pdf(np.dstack((xx, yy)), mean=mu2, cov=cov)
ax[1].contour(xx, yy, zz, colors='red')
zz = (h2(np.dstack((xx, yy)), mu1, cov, mu2, cov) < 0).astype(float)
ax[1].contourf(xx, yy, zz, cmap='bwr', alpha=0.2)
ax[1].set_xlim(xmin, xmax)
ax[1].set_xlabel('Height')
ax[1].set_ylim(ymin, ymax)
ax[1].set_ylabel('Weight')
ax[1].set_aspect('equal')
ax[1].set_title('LDA')
ax[1].legend()

fig.tight_layout()
plt.show()

左が二次判別分析，右が線形判別分析の結果です．線形判別分析では，2クラスの正規分布の分散共分散行列が等しいと仮定しているため，等高線の形や向きが等しくなっています（この例では分かりづらいですが...）．また，判別境界（$h(\pmb{x}) = 0$ となる $\pmb{x}$ が描く曲線）は，二次判別分析の場合は二次曲線ですが，線形判別分析の場合は直線となっています．

---
### 多クラスの場合の判別分析

ここまで，クラスの数を2に限定して判別分析を説明してきましたが，判別分析は多クラスの（クラス数3以上の）問題にも適用できます．


#### 方法


データを「クラス$1$」から「クラス$K$」までの $K$ 通りのクラスに判別する問題の場合，次のようにします：
1. クラス $k$ ($k = 1, 2, \ldots, K$) のデータが正規分布 $N\left(\pmb{\mu}_k, \Sigma_k\right)$ に従うと仮定して，パラメータ $\pmb{\mu}_k, \Sigma_k$ を推定する．
1. クラスが未知のデータ $\pmb{x}$ が与えられたら，それが $N\left(\pmb{\mu}_k, \Sigma_k\right)$ から得られたとすることの対数尤度 $\log\ell_k(\pmb{x})$ ($k = 1, 2, \ldots, K$) を求め，どのクラスに対して尤度が最大となるか調べる：

$$
\DeclareMathOperator*{\argmax}{argmax}
k^{*} = \argmax_{k=1,\ldots,K}{\log\ell_k(\pmb{x})} \qquad (9)
$$

ここで，$\displaystyle \DeclareMathOperator*{\argmax}{argmax} \argmax_x f(x)$ は，$f(x)$ が最大となるときの $x$ の値を返す演算子です（$\displaystyle \DeclareMathOperator*{\argmin}{argmin} \argmin$ というのもあります）．
例えば $f(x) = -(x-1)^2 + 2$ のとき，

$$
\DeclareMathOperator*{\argmax}{argmax}
\begin{aligned}
\max_{x} f(x) &= 2 & (10) \\
\argmax_{x} f(x) &= 1 & (11) \\
\end{aligned}
$$

ということになります．式$(10)$の値は，$x$ を全ての実数値の範囲で変化させたときの $f(x)$ の最大値，式$(11)$の値は，そのときの $x$ の値です．
これらのことから分かるように，式$(9)$ の $k^{*}$ は，尤度最大となったクラスの番号となります．

上記では，分散共分散行列がクラス毎に異なってもよい考えていますので，二次判別分析の説明となっています．線形判別分析の場合は，全てのクラスの分散共分散行列が共通である，つまり，
$
\Sigma_1 = \Sigma_2 = \dots = \Sigma_K = \Sigma
$
とすることになります．

#### 実験: Fisher のアヤメのデータの判別分析

前回登場した Fisher のアヤメのデータにはクラスが3つあります．判別分析してみましょう．

In [ ]:
# Fisher のアヤメのデータ
iris = load_iris(as_frame=True)
dfIris = iris.frame
dfIris

In [ ]:
## データの準備
# 被説明変数（'target'列）を除いたものを np.array に
X_iris = dfIris.drop(columns='target').to_numpy()
print('# X_iris の最初の5行と X_iris.shape')
print(X_iris[:5, :], X_iris.shape)
print()
# 'target' 列にクラス番号が格納されている
y_iris = dfIris['target'].to_numpy(dtype=int)
for k, tn in enumerate(iris.target_names):
    print(f'Class{k}: {tn}')
print()
print('# y_iris と y_iris.shape')
print(y_iris, y_iris.shape)

ここでは，[scikit-learn](https://scikit-learn.org/) にある次の二つを利用します．

- 線形判別分析: [sklearn.discriminant_analysis.LinearDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)
- 二次判別分析: [sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html)

次の二つのコードセルで判別分析を実行します．

In [ ]:
print('##### 線形判別分析 #####')
print()
N = y_iris.shape[0]
lda = LinearDiscriminantAnalysis(priors=np.ones(3)/3)
lda.fit(X_iris, y_iris)  # パラメータの推定
y_pred = lda.predict(X_iris) # 予測
print('# クラス予測結果')
print(y_pred)
print()

# 混同行列 confusion[i, j] は，正解が i 番目のクラスで予測が j 番目のクラスだったデータの数
confusion = confusion_matrix(y_iris, y_pred)
ncorrect = np.sum(np.diag(confusion))

# 混同行列を Markdown の表形式で表示
ss = f'''
|**LDA**|予測 Class0|予測 Class1|予測 Class2|
|:---:|--:|--:|--:|
|**正解 Class0**|{confusion[0, 0]}|{confusion[0, 1]}|{confusion[0, 2]}|
|**正解 Class1**|{confusion[1, 0]}|{confusion[1, 1]}|{confusion[1, 2]}|
|**正解 Class2**|{confusion[2, 0]}|{confusion[2, 1]}|{confusion[2, 2]}|

(accuracy) = {ncorrect} / {N} = {ncorrect/N:.2f}
'''
display(Markdown(ss))

In [ ]:
print('##### 二次判別分析 #####')
print()
N = y_iris.shape[0]
qda = QuadraticDiscriminantAnalysis(priors=np.ones(3)/3)
qda.fit(X_iris, y_iris)  # パラメータの推定
y_pred = qda.predict(X_iris) # 予測
print('# クラス予測結果')
print(y_pred)
print()

# 混同行列 confusion[i, j] は，正解が i 番目のクラスで予測が j 番目のクラスだったデータの数
confusion = confusion_matrix(y_iris, y_pred)
ncorrect = np.sum(np.diag(confusion))

# 混同行列を Markdown の表形式で表示
ss = f'''
|**QDA**|予測 Class0|予測 Class1|予測 Class2|
|:---:|--:|--:|--:|
|**正解 Class0**|{confusion[0, 0]}|{confusion[0, 1]}|{confusion[0, 2]}|
|**正解 Class1**|{confusion[1, 0]}|{confusion[1, 1]}|{confusion[1, 2]}|
|**正解 Class2**|{confusion[2, 0]}|{confusion[2, 1]}|{confusion[2, 2]}|

(accuracy) = {ncorrect} / {N} = {ncorrect/N:.2f}
'''
display(Markdown(ss))

「accuracy」は，全てのデータのうちクラスを正しく予測できたものの割合です．このデータでは，線形判別分析と二次判別分析の予測結果は一致しており，両者の違いは見えませんでした．notebookB 以降で他にもいろいろ実験してみましょう．